In [1]:
__author__ = 'Wzhou9'

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn import cross_validation
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn import cross_validation   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from xgboost.sklearn import XGBRegressor
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline
#%matplotlib

/Users/weijiezhou/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/weijiezhou/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
rcParams['figure.figsize'] = 12, 4

In [3]:
train =pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# Add target attribute "loss" in the test dataset and fulfill with Null
test["loss"]=np.nan

In [4]:
 joined = pd.concat([train, test])

In [5]:
def remove_st(col1,col2):
    set_1 = set(col1.unique())
    set_2 = set(col2.unique())
    rm = (set_2 - set_1).union(set_1 - set_2) 
    return rm

In [6]:
#df = pd.DataFrame([{'a': 15, 'b': 15, 'c': 5}, {'a': 20, 'b': 10, 'c': 7}, {'a': 25, 'b': 30, 'c': 9}])
#remove_st(df['a'],df['b'])


In [7]:
def filter_rm(col,rm):
    if col in rm:
        return np.nan
    return col
    

In [8]:
def handle_cat(train,test):
    #join two datasets
    joinDS = pd.concat([train, test])
    headercat = joinDS.select_dtypes(include = ['object']).columns
    #
    for i in headercat:
        #find remove set
        rm_list = remove_st(train[i], test[i])
        joinDS[i] = joinDS[i].apply(lambda x: filter_rm(x,rm_list), 1)
        joinDS[i] = pd.factorize(joined[i].values, sort=True)[0]    
    return joinDS   



In [9]:
joinDS = handle_cat(train,test)

In [10]:
train_a = joinDS[joinDS['loss'].notnull()]
test_a = joinDS[joinDS['loss'].isnull()]

In [11]:
#transform loss, input of xgboost
shift = 200
train_x = train_a.drop(['loss','id'],1)
train_label = np.log(train_a['loss'] + shift)
test_x = test_a.drop(['id','loss'],1)
test_id = test['id']

    

In [12]:
useTrainCV = True
cv_folds = 5
early_stopping_rounds = 50

In [14]:
# tunning n_estimators
xgb1 = XGBRegressor(
 learning_rate =0.08,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 #objective= 'binary:logistic',
 #nthread=4,
 scale_pos_weight=1,
 seed=2016)

In [ ]:
if useTrainCV:
    xgb_param = xgb1.get_xgb_params()    
    xgtrain = xgb.DMatrix(train_x, label=train_label)
    cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=xgb1.get_params()['n_estimators'], nfold=cv_folds,
    metrics='mae', early_stopping_rounds=early_stopping_rounds,show_stdv = True)    
    xgb1.set_params(n_estimators=cvresult.shape[0])
    
    


In [ ]:
xgb1.fit(train_x, train_label, eval_metric='mae')

In [29]:
dtrain_predictions = xgb1.predict(train_x)

print ("\nModel Report")
print ("Accuracy : %.4g" % mean_absolute_error(train_label.values, dtrain_predictions))
    


Model Report
Accuracy : 0.346


In [33]:
feat_imp = pd.Series(xgb1.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')



In [36]:
#xgtest = xgb1.DMatrix(test_x)
prediction_log = xgb1.predict(test_x)
prediction = np.exp(prediction_log) - shift

In [37]:
submission = pd.DataFrame()
submission['id'] = test_id
submission['loss'] = prediction
submission.to_csv('sub_v_Gs1.csv', index=False)

In [38]:
x = pd.read_csv("sub_v.csv")

In [39]:
x.head(5)

,id,loss
0,4,1511.634033
1,6,1973.355713
2,9,8532.917969
3,12,6023.274902
4,15,797.491760


In [ ]:
#Tune max_depth and min_child_weight

In [13]:
#learnR = xgb1.get_xgb_params()['learning_rate'] 
learnR = 0.1
#n_est = xgb1.get_xgb_params()['n_estimators'] 
n_est = 881

#param2 = xgb1.get_xgb_params
#param2

In [14]:
param_test1 = {
 'max_depth':list(range(4,14,2)),
 'min_child_weight':list(range(1,6,2))
}


In [15]:
xgb2 = XGBRegressor(learning_rate =learnR, n_estimators=n_est, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:linear', nthread=-1, scale_pos_weight=1, seed=2016)

In [16]:
gsearch1 = GridSearchCV(estimator = xgb2, param_grid = param_test1,scoring='median_absolute_error',n_jobs=-1,iid=False, cv=5, verbose = 1)


In [ ]:
gsearch1.fit(train_x.values,train_label.values)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [ ]:
#Tune subsample and colsample_bytree

In [14]:

MaxD = 8
MinCW = 1

In [15]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

In [16]:
xgb3 = XGBRegressor(learning_rate =learnR, n_estimators=n_est, max_depth=MaxD,
 min_child_weight=MinCW, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:linear', nthread=-1, scale_pos_weight=1, seed=2016)

In [17]:
gsearch3 = GridSearchCV(estimator = xgb3, param_grid = param_test3,scoring='median_absolute_error',n_jobs=-1,iid=False, cv=5, verbose = 1)


In [ ]:
gsearch3.fit(train_x.values,train_label.values)
gsearch3.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [18]:
Subs = 0.8
ColS = 0.5

In [19]:
# eta: decrease the value of eta to add more trees. If you want to see the process, then adjust
# from high to low [0.06,0.04,0.02,0.01], if you want to get the best result, use 0.01 directly

In [21]:
xgb4 = XGBRegressor(
 learning_rate =learnR,
 n_estimators=n_est,
 max_depth=MaxD,
 min_child_weight=MinCW,
 gamma=0,
 subsample=Subs,
 colsample_bytree=ColS,
 #objective= 'binary:logistic',
 #nthread=4,
 scale_pos_weight=1,
 seed=2016)

In [ ]:
xgb4.fit(train_x, train_label, eval_metric='mae')

In [ ]:
prediction_log = xgb4.predict(test_x)
prediction = np.exp(prediction_log) - shift

In [ ]:
submission = pd.DataFrame()
submission['id'] = test_id
submission['loss'] = prediction
submission.to_csv('sub_v_F.csv', index=False)